# Проект: анализ экономического развития стран и его связи с социально-демографическими показателями, а также воздействием на экологию. Сравнительное исследование стран (1985 – 2023 годы)

**Цель проекта:** выявить взаимосвязь между уровнем экономического развития страны (ВВП на душу населения) и определенными социально-демографическими показателями, а также влиянием на экологию (выбросы СО2 на душу населения), проанализировать как эта зависимость менялась с течением времени для разных групп стран.

**Исходные данные:** для анализа используются данные Всемирного банка (World Bank)

**План проекта:**

1. Получить данные через API Всемирного банка и провести их предобработку

2. Загрузить полученные данные в облачную СУБД (PostgreSQL)

3. Провести исследовательский анализ полученных данных, в т.ч.:

- Построить матрицу корреляции
- Визуализировать данные и тренды (в т.ч. по группам стран, в динамике)
- Провести кластеризацию стран
  
4. Сделать выводы

В данном файле выполнены п.1 и 2. Исследовательский анализ данных в отдельном .ipynb файле

In [88]:
# импорт бтблиотек
import requests
import pandas as pd
import numpy as np
from typing import List, Optional
import sqlalchemy
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os


In [89]:
#pip install python-dotenv sqlalchemy psycopg2

## 1 Выгрузка данных через World Bank API

### 1.1 Выгрузка стран и индикаторов

In [92]:
def fetch_data(parametr):
    '''
    Функция получает данные о странах/индикаторах из API Всемирного банка.
    Параметры (str): 'country' - для выгрузки стран
                     'indicator' - для выгрузки индикаторов 
      
    Возвращает: pandas.DataFrame с данными 
    '''

    base_url = "https://api.worldbank.org/v2" # endpoint API 
    url = f"{base_url}/{parametr}"
    params = { 'format': 'json',
                'per_page': 30000  # Большое значение для получения всех данных
           }
      
    # Выполняем запрос к API
    response = requests.get(url, params=params)
    
    # обработка ошибок
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Ошибка при запросе: {response.status_code}")
        print(f"Текст ошибки: {response.text}")   

    # формируем датафрейм из полученных данных 
    df_raw = pd.DataFrame(data[1])
    
    # проверка на пустоту
    if df_raw.empty:
        print("Нет данных")
        return None  
    
    return df_raw   

In [93]:
countries_raw = fetch_data('country')

countries_raw.head()


,id,iso2Code,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
0,ABW,AW,Aruba,"{'id': 'LCN', 'iso2code': 'ZJ', 'value': 'Lati...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'HIC', 'iso2code': 'XD', 'value': 'High...","{'id': 'LNX', 'iso2code': 'XX', 'value': 'Not ...",Oranjestad,-70.0167,12.5167
1,AFE,ZH,Africa Eastern and Southern,"{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': 'Aggregates'}",,,
2,AFG,AF,Afghanistan,"{'id': 'MEA', 'iso2code': 'ZQ', 'value': 'Midd...","{'id': 'MNA', 'iso2code': 'XQ', 'value': 'Midd...","{'id': 'LIC', 'iso2code': 'XM', 'value': 'Low ...","{'id': 'IDX', 'iso2code': 'XI', 'value': 'IDA'}",Kabul,69.1761,34.5228
3,AFR,A9,Africa,"{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': 'Aggregates'}",,,
4,AFW,ZI,Africa Western and Central,"{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': 'Aggregates'}",,,


In [94]:
indicators_raw = fetch_data('indicator')
indicators_raw.head()

,id,name,unit,source,sourceNote,sourceOrganization,topics
0,1.0.HCount.1.90usd,Poverty Headcount ($1.90 a day),,"{'id': '37', 'value': 'LAC Equity Lab'}",The poverty headcount index measures the propo...,LAC Equity Lab tabulations of SEDLAC (CEDLAS a...,"[{'id': '11', 'value': 'Poverty '}]"
1,1.0.HCount.2.5usd,Poverty Headcount ($2.50 a day),,"{'id': '37', 'value': 'LAC Equity Lab'}",The poverty headcount index measures the propo...,LAC Equity Lab tabulations of SEDLAC (CEDLAS a...,"[{'id': '11', 'value': 'Poverty '}]"
2,1.0.HCount.Mid10to50,Middle Class ($10-50 a day) Headcount,,"{'id': '37', 'value': 'LAC Equity Lab'}",The poverty headcount index measures the propo...,LAC Equity Lab tabulations of SEDLAC (CEDLAS a...,"[{'id': '11', 'value': 'Poverty '}]"
3,1.0.HCount.Ofcl,Official Moderate Poverty Rate-National,,"{'id': '37', 'value': 'LAC Equity Lab'}",The poverty headcount index measures the propo...,LAC Equity Lab tabulations of data from Nation...,"[{'id': '11', 'value': 'Poverty '}]"
4,1.0.HCount.Poor4uds,Poverty Headcount ($4 a day),,"{'id': '37', 'value': 'LAC Equity Lab'}",The poverty headcount index measures the propo...,LAC Equity Lab tabulations of SEDLAC (CEDLAS a...,"[{'id': '11', 'value': 'Poverty '}]"


### 1.2 Предобработка данных 

Предобработка данных по странам

In [97]:
def countries_clearing(df_raw):
    '''
    Выполняет предобработку данных
    Параметры: pandas.DataFrame 
    Возвращает: обработанный pandas.DataFrame
    '''
    # выделяем идентификатор региона из столбца 
    df_raw['region_id'] = df_raw.region.apply(lambda x: x.get('id', None))
    
    # аналогично выделяем информацию из других столбцов
    for col in ['region', 'adminregion', 'incomeLevel', 'lendingType']:
        df_raw[col + '_value'] = df_raw[col].apply(lambda cell: cell.get('value', None))

    # оставляем  нужные для анализа столбцы и переименовываем их
    df = df_raw[['id', 'iso2Code', 'name', 'capitalCity',
                     'region_id', 'region_value', 'incomeLevel_value',
                     'lendingType_value']]
    
    df = df.rename(columns={'id':'id_cnt',
                            'iso2Code':'iso2_code',
                            'capitalCity':'capital_city',
                            'incomeLevel_value':'income_level_value',
                            'lendingType_value':'lending_type_value'})
    # удаляем дубликаты строк
    df = df.drop_duplicates()
    
    # удаляем аггрегированные регионы 
    df = df.loc[df['region_value']!='Aggregates']
  
    return df


In [98]:
countries = countries_clearing(countries_raw)
countries.info()
countries.head()

<class 'pandas.core.frame.DataFrame'>
Index: 217 entries, 0 to 295
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_cnt              217 non-null    object
 1   iso2_code           217 non-null    object
 2   name                217 non-null    object
 3   capital_city        217 non-null    object
 4   region_id           217 non-null    object
 5   region_value        217 non-null    object
 6   income_level_value  217 non-null    object
 7   lending_type_value  217 non-null    object
dtypes: object(8)
memory usage: 15.3+ KB


,id_cnt,iso2_code,name,capital_city,region_id,region_value,income_level_value,lending_type_value
0,ABW,AW,Aruba,Oranjestad,LCN,Latin America & Caribbean,High income,Not classified
2,AFG,AF,Afghanistan,Kabul,MEA,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA
5,AGO,AO,Angola,Luanda,SSF,Sub-Saharan Africa,Lower middle income,IBRD
6,ALB,AL,Albania,Tirane,ECS,Europe & Central Asia,Upper middle income,IBRD
7,AND,AD,Andorra,Andorra la Vella,ECS,Europe & Central Asia,High income,Not classified


Выгрузили данные по 296 странам, пропусков нет, тип данных корректный  и не требует изменений


In [100]:
# формируем список стран для дальнейшего использования
countries_list = countries['id_cnt'].tolist()
countries_list

['ABW',
 'AFG',
 'AGO',
 'ALB',
 'AND',
 'ARE',
 'ARG',
 'ARM',
 'ASM',
 'ATG',
 'AUS',
 'AUT',
 'AZE',
 'BDI',
 'BEL',
 'BEN',
 'BFA',
 'BGD',
 'BGR',
 'BHR',
 'BHS',
 'BIH',
 'BLR',
 'BLZ',
 'BMU',
 'BOL',
 'BRA',
 'BRB',
 'BRN',
 'BTN',
 'BWA',
 'CAF',
 'CAN',
 'CHE',
 'CHI',
 'CHL',
 'CHN',
 'CIV',
 'CMR',
 'COD',
 'COG',
 'COL',
 'COM',
 'CPV',
 'CRI',
 'CUB',
 'CUW',
 'CYM',
 'CYP',
 'CZE',
 'DEU',
 'DJI',
 'DMA',
 'DNK',
 'DOM',
 'DZA',
 'ECU',
 'EGY',
 'ERI',
 'ESP',
 'EST',
 'ETH',
 'FIN',
 'FJI',
 'FRA',
 'FRO',
 'FSM',
 'GAB',
 'GBR',
 'GEO',
 'GHA',
 'GIB',
 'GIN',
 'GMB',
 'GNB',
 'GNQ',
 'GRC',
 'GRD',
 'GRL',
 'GTM',
 'GUM',
 'GUY',
 'HKG',
 'HND',
 'HRV',
 'HTI',
 'HUN',
 'IDN',
 'IMN',
 'IND',
 'IRL',
 'IRN',
 'IRQ',
 'ISL',
 'ISR',
 'ITA',
 'JAM',
 'JOR',
 'JPN',
 'KAZ',
 'KEN',
 'KGZ',
 'KHM',
 'KIR',
 'KNA',
 'KOR',
 'KWT',
 'LAO',
 'LBN',
 'LBR',
 'LBY',
 'LCA',
 'LIE',
 'LKA',
 'LSO',
 'LTU',
 'LUX',
 'LVA',
 'MAC',
 'MAF',
 'MAR',
 'MCO',
 'MDA',
 'MDG',
 'MDV',


Предобработка данных по индикаторам

In [102]:
def indicators_clearing(df_raw):
    '''
    Выполняет предобработку данных
    Параметры: pandas.DataFrame 
    Возвращает: обработанный pandas.DataFrame
    '''
    # выделяем значение источника из столбца 
    df_raw['source'] = df_raw.source.apply(lambda x: x.get('value', None))

    # оставляем  нужные для анализа столбцы 
    df = df_raw[['id', 'name', 'source','sourceNote']]

    df = df.rename(columns={'id':'id_ind','sourceNote':'source_note'})
    
    return df 


In [103]:
indicators = indicators_clearing(indicators_raw)
indicators.info()
indicators.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29312 entries, 0 to 29311
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_ind       29312 non-null  object
 1   name         29312 non-null  object
 2   source       29312 non-null  object
 3   source_note  29312 non-null  object
dtypes: object(4)
memory usage: 916.1+ KB


,id_ind,name,source,source_note
0,1.0.HCount.1.90usd,Poverty Headcount ($1.90 a day),LAC Equity Lab,The poverty headcount index measures the propo...
1,1.0.HCount.2.5usd,Poverty Headcount ($2.50 a day),LAC Equity Lab,The poverty headcount index measures the propo...
2,1.0.HCount.Mid10to50,Middle Class ($10-50 a day) Headcount,LAC Equity Lab,The poverty headcount index measures the propo...
3,1.0.HCount.Ofcl,Official Moderate Poverty Rate-National,LAC Equity Lab,The poverty headcount index measures the propo...
4,1.0.HCount.Poor4uds,Poverty Headcount ($4 a day),LAC Equity Lab,The poverty headcount index measures the propo...


Выгрузили список всех индикаторов с описанием, пропусков нет, тип данных корректный. 

In [105]:
# формируем список показателей, с которыми в дальнейшем будем работать
indicators_list = [
    'SP.POP.TOTL',
    'SP.POP.GROW',
    'SL.UEM.TOTL.NE.ZS',
    'SP.DYN.LE00.IN',
    'SI.POV.NAHC',
    'FP.CPI.TOTL.ZG',
    'NY.GDP.MKTP.CD',
    'NY.GDP.MKTP.KD.ZG',
    'NV.AGR.TOTL.ZS',
    'NV.IND.TOTL.ZS',
    'EG.USE.ELEC.KH.PC',
    'EG.ELC.ACCS.ZS',
    'EN.GHG.CO2.ZG.AR5',
    'EN.GHG.CO2.PC.CE.AR5',
    'IT.NET.USER.ZS',
    'SE.XPD.TOTL.GD.ZS',
    'GB.XPD.RSDV.GD.ZS',
    'BX.KLT.DINV.CD.WD',
    'NY.GDP.PCAP.CD',
    'NY.GDP.PCAP.KD.ZG',
    'NY.GDP.PCAP.PP.CD',
    'SH.ALC.PCAP.LI',
    'NY.GNP.ATLS.CD',
    'NY.GNP.PCAP.CD',
    'SI.POV.GINI',
    'SP.URB.TOTL.IN.ZS'
]


### 1.3 Выгрузка значений выбранных показателей за период по всем странам

In [107]:
def fetch_worldbank_data(indicators: List[str],
                         countries: List[str],
                         start_year: int,
                         end_year: int,
                         language: str = 'en') -> Optional[pd.DataFrame]:

    '''
    Функция получает данные показателей из API Всемирного банка.

    Параметры:
        indicators: Список кодов показателей 
        countries: Список кодов стран в формате ISO 2
        start_year: Год начала периода
        end_year: Год окончания периода
        language: Язык данных 

    Возвращает:
        pandas.DataFrame с данными показателей
    '''

    base_url = "https://api.worldbank.org/v2" # endpoint API 

    # преобразовываем страны в строку с разделителем точка с запятой (для запроса данных)
    countries_str = ';'.join(countries)

    # список для хранения данных о показателях
    all_data = []

    try:
        for indicator in indicators:
            # Формируем URL для запроса
            url = f"{base_url}/{language}/country/{countries_str}/indicator/{indicator}"
            params = {
                'format': 'json',
                'date': f"{start_year}:{end_year}",
                'per_page': 10000  # Большое значение для получения всех данных
            }

            # Выполняем запрос к API
            response = requests.get(url, params=params)
            
            if response.status_code == 404:
                    print(f"Данные не найдены для индикатора {indicator}")
                    break
            
            response.raise_for_status()

            data = response.json()

            # API возвращает массив, где первый элемент - метаданные, второй - данные
            if len(data) > 1 and isinstance(data[1], list):
                for item in data[1]:
                   
                    all_data.append({
                            'country': item['country']['value'],
                            'iso3_code': item['countryiso3code'],
                            'indicator': item['indicator']['value'],
                            'indicator_code': item['indicator']['id'],
                            'year': int(item['date']),
                            'value': item['value']
                        })

        # Создаем DataFrame
        df = pd.DataFrame(all_data)

        if df.empty:
            print("Предупреждение: Не получено данных для указанных параметров")
            return None

        return df

    # Обрабатываем возможные ошибки при работе с АПИ
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к API: {e}")
        return None
    except (KeyError, IndexError, ValueError, TypeError) as e:
        print(f"Ошибка при обработке данных: {e}")
        return None

In [108]:
# выгружаем данные за выбранный период по предварительно сформированному списку показателей и  стран
data = fetch_worldbank_data(
    indicators = indicators_list,
    countries = countries_list,
    start_year = '1985',
    end_year = '2024',
    ) 
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225680 entries, 0 to 225679
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   country         225680 non-null  object 
 1   iso3_code       225680 non-null  object 
 2   indicator       225680 non-null  object 
 3   indicator_code  225680 non-null  object 
 4   year            225680 non-null  int64  
 5   value           164591 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.3+ MB


,country,iso3_code,indicator,indicator_code,year,value
0,Aruba,ABW,"Population, total",SP.POP.TOTL,2024,107624.0
1,Aruba,ABW,"Population, total",SP.POP.TOTL,2023,107359.0
2,Aruba,ABW,"Population, total",SP.POP.TOTL,2022,107310.0
3,Aruba,ABW,"Population, total",SP.POP.TOTL,2021,107700.0
4,Aruba,ABW,"Population, total",SP.POP.TOTL,2020,108587.0


Выгружены значения выбранных показателей по всем странам за период 1985-2024, типы данных корркетны, имеются пропуски в значениях (т.к. не за все года есть данные), на данном этапе пропуски остаются. При необходимости обработка пропусков будет сделана на этапе анализа. 


## 2. Подключение и загрузка данных в базу данных

Подключаемся к базе Supabase

In [112]:
# Load environment variables from .env
load_dotenv()

# Fetch variables
USER = os.getenv("user", 'postgres.rfhnmbttgskiamdmzjyh')
PASSWORD = os.getenv("password",'pst484971')
HOST = os.getenv("host",'aws-1-eu-central-1.pooler.supabase.com')
PORT = "6543"
DBNAME = os.getenv("dbname",'postgres')

# Construct the SQLAlchemy connection string
DATABASE_URL = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}?sslmode=require"

# Create the SQLAlchemy engine
engine = create_engine(DATABASE_URL) 

# Test the connection
try:
    with engine.connect() as connection:
        print("Connection successful!")
except Exception as e:
    print(f"Failed to connect: {e}")


Connection successful!


In [113]:
# cохраняем данные в базу данных 
countries.to_sql('countries', con=engine, if_exists='replace')
indicators.to_sql('indicators', con=engine, if_exists='replace')
data.to_sql('data', con=engine, if_exists='replace')


680

In [114]:
# проверка запросами
pd.read_sql_query('SELECT * FROM countries limit 5', con=engine)


,index,id_cnt,iso2_code,name,capital_city,region_id,region_value,income_level_value,lending_type_value
0,0,ABW,AW,Aruba,Oranjestad,LCN,Latin America & Caribbean,High income,Not classified
1,2,AFG,AF,Afghanistan,Kabul,MEA,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA
2,5,AGO,AO,Angola,Luanda,SSF,Sub-Saharan Africa,Lower middle income,IBRD
3,6,ALB,AL,Albania,Tirane,ECS,Europe & Central Asia,Upper middle income,IBRD
4,7,AND,AD,Andorra,Andorra la Vella,ECS,Europe & Central Asia,High income,Not classified


In [115]:
pd.read_sql_query('SELECT * FROM indicators limit 5', con=engine)

,index,id_ind,name,source,source_note
0,0,1.0.HCount.1.90usd,Poverty Headcount ($1.90 a day),LAC Equity Lab,The poverty headcount index measures the propo...
1,1,1.0.HCount.2.5usd,Poverty Headcount ($2.50 a day),LAC Equity Lab,The poverty headcount index measures the propo...
2,2,1.0.HCount.Mid10to50,Middle Class ($10-50 a day) Headcount,LAC Equity Lab,The poverty headcount index measures the propo...
3,3,1.0.HCount.Ofcl,Official Moderate Poverty Rate-National,LAC Equity Lab,The poverty headcount index measures the propo...
4,4,1.0.HCount.Poor4uds,Poverty Headcount ($4 a day),LAC Equity Lab,The poverty headcount index measures the propo...


In [116]:
pd.read_sql_query('SELECT * FROM data limit 5', con=engine)

,index,country,iso3_code,indicator,indicator_code,year,value
0,0,Aruba,ABW,"Population, total",SP.POP.TOTL,2024,107624.0
1,1,Aruba,ABW,"Population, total",SP.POP.TOTL,2023,107359.0
2,2,Aruba,ABW,"Population, total",SP.POP.TOTL,2022,107310.0
3,3,Aruba,ABW,"Population, total",SP.POP.TOTL,2021,107700.0
4,4,Aruba,ABW,"Population, total",SP.POP.TOTL,2020,108587.0


**Вывод**: необходимые данные выгружены в облачную СУБД, выполнена предобработка для дальнейшего использования